In [2]:
import pandas as pd

file_name = 'results/지하철_위도경도.csv'
df = pd.read_csv(file_name)

df = df[['전철역명', '구']]
df = df.set_index('전철역명')

df

,구
전철역명,
석계,노원구
서울,중구
영등포,영등포구
대방,영등포구
신도림,구로구
...,...
정릉,성북구
북한산보국문,성북구
신설동,성북구


In [3]:
import geopandas as gpd
from shapely.geometry import Point

# 서울시 행정구역 GeoJSON 데이터를 로드
geojson_path = 'data/HangJeongDong_ver20230701.geojson'
districts = gpd.read_file(geojson_path)

# 지하철 역 데이터를 GeoDataFrame으로 변환
df['geometry'] = df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
subway_gdf = gpd.GeoDataFrame(df, geometry='geometry')

# 좌표계 통일 (서울시 행정구역 GeoJSON 데이터와 동일한 좌표계를 사용해야 함)
subway_gdf.set_crs(epsg=4326, inplace=True)  # WGS84 좌표계

# 각 역의 좌표가 속한 구 찾기 (op 대신 predicate 사용)
subway_gdf = gpd.sjoin(subway_gdf, districts, how='left', predicate='within')

# 결과 데이터프레임에서 필요한 열만 선택
result_df = subway_gdf[['위도', '경도', 'sggnm']].copy()
result_df.rename(columns={'sggnm': '구'}, inplace=True)

# 인덱스로 그룹화하고 평균 계산
grouped_df = result_df.groupby(result_df.index).agg({
    '위도': 'mean',
    '경도': 'mean',
    '구': 'first'  # '구' 정보는 첫 번째 값을 사용
})

# 결과 출력
grouped_df

KeyError: '경도'